2021_06/1

In [1]:
import numpy as np
import ray
import time
from tqdm import tqdm

In [2]:
with open('data/2021_06.txt') as f:
    data = np.array(list((map(int, f.readlines()[0].split(',')))))

In [3]:
@ray.remote
def simulate(data, number_of_days):
        
    for day in range(1,number_of_days+1):
        
        data = data - 1
        if (data<0).any() == True:
            negative = np.where(data<0)[0]
            data[negative] = 6
            data = np.append(data,[8]*len(negative))
    
    return len(data)

In [7]:
try:
    ray.shutdown()
    time.sleep(1)
except:
    pass

num_cpus = 4
ray.init(num_cpus=num_cpus,ignore_reinit_error=True)
time.sleep(1)

2022-12-02 09:07:03,191	INFO resource_spec.py:231 -- Starting Ray with 7.71 GiB memory available for workers and up to 3.87 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2022-12-02 09:07:03,431	INFO services.py:1193 -- View the Ray dashboard at localhost:8265
2022-12-02 09:07:06,991	WARNING worker.py:1134 -- The dashboard on node IWT-PC-24 failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mrbenkoe\Miniconda3\envs\ds_env\lib\site-packages\ray\dashboard/dashboard.py", line 960, in <module>
    metrics_export_address=metrics_export_address)
  File "C:\Users\mrbenkoe\Miniconda3\envs\ds_env\lib\site-packages\ray\dashboard/dashboard.py", line 513, in __init__
    build_dir = setup_static_dir(self.app)
  File "C:\Users\mrbenkoe\Miniconda3\envs\ds_env\lib\site-packages\ray\dashboard/dashboard.py", line 414, in setup_static_dir
    "&& npm run build)", build_dir)
FileNotFoundError: [Errno 2] Dashboar

In [8]:
%%time

days = 80

input_list = []
for dataset in np.array_split(data, num_cpus):
    input_list.append(simulate.remote(dataset, days))

output_list = []
for ipt in input_list:
    output_list.append(ray.get([ipt]))
    
ray.shutdown()

print('Result = ',sum(np.array(output_list).flatten())) # 358214 [t = 3.73 s]

Result =  358214
Wall time: 3.73 s


2021_06/2

...to slow.